In [9]:
import pandas as pd
from tensorflow.keras import models
from sklearn.preprocessing import MinMaxScaler , LabelEncoder , OneHotEncoder , StandardScaler

In [13]:
model = models.load_model('./ck7/weights_val-0.8420_train-0.8340__ep-48770.hdf5')

In [14]:
train = pd.read_csv('../input/train.csv', index_col='id').sample(frac = 0.1, )

unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}
        
        
scaler = StandardScaler()
labels = train['type']
train = train.drop(columns=['fiberID', 'type']) 
        
_mat = scaler.fit_transform(train)
train = pd.DataFrame(_mat, columns=train.columns, index=train.index)
        
train_x = train
train_y = labels.replace(label_dict)  

In [15]:
test = pd.read_csv('../input/test.csv').reset_index(drop=True)
test_ids = test['id']
test = test.drop(columns=['id', 'fiberID'])
test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

In [16]:
pred_mat = model.predict(test)

sample = pd.read_csv('../input/sample_submission.csv')

submission = pd.DataFrame(pred_mat, index=test.index)
submission = submission.rename(columns=i2lb)
submission = pd.concat([test_ids, submission], axis=1)
submission = submission[sample.columns]
submission.to_csv("./sub/ck7.csv", index=False)